In [1]:
import numpy as np

print("="*80)
print("Problem 1: u''(x) = f(x), u(0)=0, u(1)=0")
print("f(x) = 1 for 0.4 ≤ x ≤ 0.6, else 0")
print("="*80)

def f(x):
    """Right-hand side."""
    return np.where((x >= 0.4) & (x <= 0.6), 1.0, 0.0)

def u_exact(x):
    """
    CORRECT exact solution derived by integration:
    
    u''(x) = f(x)
    Integrate once: u'(x) = C₁ + ∫₀ˣ f(t) dt
    Integrate twice: u(x) = C₁x + C₂ + ∫₀ˣ ∫₀ˢ f(t) dt ds
    
    With u(0)=0, u(1)=0:
    C₂ = 0
    C₁ = -∫₀¹ (1-t)f(t) dt = -∫₀.₄⁰·⁶ (1-t) dt = -0.1
    
    Results in piecewise solution:
    Region 1 (0 ≤ x < 0.4):    u(x) = -0.1·x
    Region 2 (0.4 ≤ x ≤ 0.6):  u(x) = -0.1·x + 0.5·(x-0.4)²
    Region 3 (0.6 < x ≤ 1):     u(x) = -0.1·(1-x)
    """
    x = np.asarray(x, dtype=float)
    u = np.zeros_like(x)
    
    # Region 1: 0 ≤ x < 0.4
    mask1 = (x < 0.4)
    u[mask1] = -0.1 * x[mask1]
    
    # Region 2: 0.4 ≤ x ≤ 0.6
    mask2 = (0.4 <= x) & (x <= 0.6)
    u[mask2] = -0.1 * x[mask2] + 0.5 * (x[mask2] - 0.4)**2
    
    # Region 3: 0.6 < x ≤ 1
    mask3 = (x > 0.6)
    u[mask3] = -0.1 * (1.0 - x[mask3])
    
    return u

def solve_fdm(N):
    """Finite-difference solver for u'' = f, u(0)=0, u(1)=0."""
    h = 1.0 / N
    x = np.linspace(0, 1, N+1)
    
    M = N - 1
    diag = -2.0 / h**2 * np.ones(M)
    off = 1.0 / h**2 * np.ones(M - 1)
    A = np.diag(diag) + np.diag(off, 1) + np.diag(off, -1)
    
    rhs = f(x[1:-1])
    u_inner = np.linalg.solve(A, rhs)
    
    u = np.zeros(N + 1)
    u[1:-1] = u_inner
    return x, u

# Verify exact solution
print("\nVerifying exact solution:")
print(f"u(0) = {u_exact(0.0):.15f} (should be 0)")
print(f"u(1) = {u_exact(1.0):.15f} (should be 0)")
print(f"u(0.4) = {u_exact(0.4):.15f}")
print(f"u(0.6) = {u_exact(0.6):.15f}")

print("\nChecking continuity at x=0.4:")
u_left_4 = -0.1 * 0.4
u_right_4 = -0.1 * 0.4 + 0.5 * (0.4 - 0.4)**2
print(f"u(0.4⁻) = {u_left_4:.15f}")
print(f"u(0.4⁺) = {u_right_4:.15f}")

print("\nChecking continuity at x=0.6:")
u_left_6 = -0.1 * 0.6 + 0.5 * (0.6 - 0.4)**2
u_right_6 = -0.1 * (1.0 - 0.6)
print(f"u(0.6⁻) = {u_left_6:.15f}")
print(f"u(0.6⁺) = {u_right_6:.15f}")

print("\n" + "="*80)
print("Convergence Study")
print("="*80)
print(f"{'N':>6} {'h':>12} {'L∞ Error':>14} {'L2 Error':>14}")
print("-"*55)

Ns = [20, 40, 80, 160, 320, 640]
for N in Ns:
    x, u_num = solve_fdm(N)
    u_ex = u_exact(x)
    err_inf = np.max(np.abs(u_num - u_ex))
    err_l2 = np.sqrt(np.mean((u_num - u_ex)**2))
    h = 1.0 / N
    print(f"{N:6d} {h:12.4e} {err_inf:14.6e} {err_l2:14.6e}")

Problem 1: u''(x) = f(x), u(0)=0, u(1)=0
f(x) = 1 for 0.4 ≤ x ≤ 0.6, else 0

Verifying exact solution:
u(0) = -0.000000000000000 (should be 0)
u(1) = -0.000000000000000 (should be 0)
u(0.4) = -0.040000000000000
u(0.6) = -0.040000000000000

Checking continuity at x=0.4:
u(0.4⁻) = -0.040000000000000
u(0.4⁺) = -0.040000000000000

Checking continuity at x=0.6:
u(0.6⁻) = -0.040000000000000
u(0.6⁺) = -0.040000000000000

Convergence Study
     N            h       L∞ Error       L2 Error
-------------------------------------------------------
    20   5.0000e-02   2.000000e-03   1.144344e-03
    40   2.5000e-02   1.000000e-03   5.724892e-04
    80   1.2500e-02   5.000000e-04   2.871677e-04
   160   6.2500e-03   2.500000e-04   1.439237e-04
   320   3.1250e-03   1.250000e-04   7.206068e-05
   640   1.5625e-03   6.250000e-05   3.605678e-05
